In [ ]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import concurrent.futures
import time
import random
import threading
import pandas as pd

stop_flag = threading.Event()

def fetch_fight_details(event_info):
    event_link, event_name = event_info
    fight_details = []

    while not stop_flag.is_set():
        try:
            page = urlopen(event_link)
            html = page.read().decode("utf-8")
            soup = BeautifulSoup(html, "html.parser")
            fights = soup.find_all(class_='b-fight-details__table-row')[1:]

            for fight in fights:
                if stop_flag.is_set():
                    return fight_details

                fight_link = fight["data-link"]
                fight_page = urlopen(fight_link)
                fight_html = fight_page.read().decode("utf-8")
                fight_soup = BeautifulSoup(fight_html, "html.parser")

                fighter_1 = fight_soup.find_all(class_='b-fight-details__person-link')[0].text.strip()
                fighter_1_link = fight_soup.find_all(class_='b-fight-details__person-link')[0].get("href")
                fighter_1_result = fight_soup.find_all(class_="b-fight-details__person-status")[0].text.strip()

                fighter_2 = fight_soup.find_all(class_='b-fight-details__person-link')[1].text.strip()
                fighter_2_link = fight_soup.find_all(class_='b-fight-details__person-link')[1].get("href")
                fighter_2_result = fight_soup.find_all(class_="b-fight-details__person-status")[1].text.strip()

                weight_class = fight_soup.find_all(class_='b-fight-details__fight-title')[0].text.strip()
                method = fight_soup.find_all(class_='b-fight-details__label')[0].find_next_sibling().text.strip()

                info = fight_soup.find_all(class_='b-fight-details__text-item')
                end_round = info[0].text.strip()[-1]
                end_time_round = info[1].text.strip()[-4:]
                end_time_round = int(end_time_round.split(":")[0]) * 60 + int(end_time_round.split(":")[1])
                max_round = info[2].text.strip()[12:].strip()[0] if info[2].text.strip()[12:] else ""
                referee = info[3].text.strip()[20:].strip()

                fight_details.append([event_name, fighter_1_link, fighter_1, fighter_1_result,
                                      fighter_2_link, fighter_2, fighter_2_result, weight_class,
                                      method, end_round, end_time_round, max_round, referee, fight_link])

            return fight_details

        except Exception as e:
            if stop_flag.is_set():
                return fight_details
            print(f"Error fetching fight details for {event_name}: {e}")
            time.sleep(random.uniform(2, 5))

# Read events from CSV
all_events_link = []
all_events_name = []

with open('events.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        all_events_link.append(row[6])
        all_events_name.append(row[1])

event_name_and_link = list(zip(all_events_link[1:], all_events_name[1:]))

fight_id = 0
processed = 0

with open('fight_overview.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    field = ["fight_id", "event_name", "fighter_1_link", "fighter_1", "fighter_1_result",
             "fighter_2_link", "fighter_2", "fighter_2_result", "bout_type", "method",
             "end_round", "end_time_round", "max_round", "referee", "fight_link"]
    writer.writerow(field)

    futures = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        for event_info in event_name_and_link:
            future = executor.submit(fetch_fight_details, event_info)
            futures.append(future)

        for future in futures:
            fight_details = future.result()
            for detail in fight_details:
                if stop_flag.is_set():
                    break
                fight_id += 1
                processed += 1
                writer.writerow([fight_id] + detail)
                print(f"Processed fight: {processed}")

print(f"Total fights processed: {processed}")